In [138]:
import numpy as np
import pandas as pd
import iisignature
from iisignature import sig, prepare, logsig, logsiglength
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [139]:
x_train = pd.read_csv("/Users/runminghuang/Desktop/capstone/Data/x_train.csv")
y_train = pd.read_csv("/Users/runminghuang/Desktop/capstone/Data/y_train.csv")
x_test = pd.read_csv("/Users/runminghuang/Desktop/capstone/Data/x_test.csv")
y_test = pd.read_csv("/Users/runminghuang/Desktop/capstone/Data/test_rand.csv")

In [140]:
# x_train = pd.read_csv("/Users/ja/Transfer_Learning_for_Stock_Prediction/x_train.csv")

In [141]:
x_train.sort_values(by=['STOCK', "ID"]).reset_index(drop=True)

,ID,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,...,RET_16,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20
0,2377,1,0,37,12,5,94,-0.005967,0.136699,0.009031,...,-0.009219,-0.493354,-0.007660,-0.585497,-0.001063,-0.351363,0.005127,-0.324675,-0.019275,-0.291751
1,5198,4,0,37,12,5,94,0.001348,-0.269520,0.011100,...,-0.000762,-0.313575,0.007867,0.071338,0.007733,-0.405243,-0.003276,-0.424336,-0.010489,-0.050591
2,8017,5,0,37,12,5,94,-0.014405,0.192655,0.003614,...,0.002742,-0.367499,-0.005843,-0.405562,0.002930,-0.315935,0.010462,-0.474957,-0.003541,-0.260130
3,20826,11,0,37,12,5,94,0.008938,0.430916,0.002662,...,0.009097,0.023598,0.011266,0.079711,0.019038,-0.230167,-0.000287,-0.312123,0.008682,-0.226628
4,33843,21,0,37,12,5,94,-0.006523,-0.060371,-0.007632,...,-0.014461,-0.337686,-0.007224,-0.161117,-0.001461,-0.095494,0.012667,0.471895,-0.038752,1.532045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418590,391556,206,5716,50,17,7,114,0.029552,-0.075091,-0.001428,...,0.024553,0.076480,0.017026,0.170516,0.002276,-0.106224,-0.034597,0.123750,-0.015676,-0.228186
418591,394490,208,5716,50,17,7,114,0.008316,0.028099,-0.006688,...,0.020970,0.090287,0.002887,-0.050408,0.008736,-0.159294,0.027350,-0.022922,0.008186,-0.080569
418592,400150,210,5716,50,17,7,114,-0.004633,-0.173518,0.001687,...,-0.024517,0.695373,-0.015320,-0.149467,-0.035810,-0.262389,0.000896,-0.172450,0.008586,-0.482171
418593,403129,211,5716,50,17,7,114,0.010883,0.172313,0.008844,...,0.005832,-0.045512,-0.008823,-0.026153,-0.011428,-0.142636,0.011253,-0.224195,0.000609,-0.341878


In [142]:
df = x_train.sort_values(by=['STOCK', "ID"]).reset_index(drop=True)
df = df.drop(columns=['DATE'])
df = df.fillna(0)

ret_cols = [col for col in df.columns if col.startswith('RET')]
df[ret_cols] = df[ret_cols].apply(lambda x: np.log1p(x))

df

,ID,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,VOLUME_2,...,RET_16,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20
0,2377,0,37,12,5,94,-0.005985,0.136699,0.008990,-0.003109,...,-0.009262,-0.493354,-0.007689,-0.585497,-0.001064,-0.351363,0.005114,-0.324675,-0.019463,-0.291751
1,5198,0,37,12,5,94,0.001347,-0.269520,0.011039,-0.085622,...,-0.000762,-0.313575,0.007837,0.071338,0.007704,-0.405243,-0.003281,-0.424336,-0.010544,-0.050591
2,8017,0,37,12,5,94,-0.014510,0.192655,0.003607,0.353804,...,0.002739,-0.367499,-0.005860,-0.405562,0.002926,-0.315935,0.010408,-0.474957,-0.003547,-0.260130
3,20826,0,37,12,5,94,0.008898,0.430916,0.002659,0.175444,...,0.009056,0.023598,0.011203,0.079711,0.018859,-0.230167,-0.000287,-0.312123,0.008644,-0.226628
4,33843,0,37,12,5,94,-0.006544,-0.060371,-0.007661,-0.037806,...,-0.014567,-0.337686,-0.007250,-0.161117,-0.001462,-0.095494,0.012588,0.471895,-0.039523,1.532045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418590,391556,5716,50,17,7,114,0.029123,-0.075091,-0.001429,-0.026934,...,0.024256,0.076480,0.016883,0.170516,0.002273,-0.106224,-0.035209,0.123750,-0.015800,-0.228186
418591,394490,5716,50,17,7,114,0.008282,0.028099,-0.006711,-0.169174,...,0.020753,0.090287,0.002883,-0.050408,0.008698,-0.159294,0.026983,-0.022922,0.008153,-0.080569
418592,400150,5716,50,17,7,114,-0.004644,-0.173518,0.001686,-0.231339,...,-0.024822,0.695373,-0.015439,-0.149467,-0.036467,-0.262389,0.000896,-0.172450,0.008549,-0.482171
418593,403129,5716,50,17,7,114,0.010824,0.172313,0.008805,-0.198001,...,0.005815,-0.045512,-0.008862,-0.026153,-0.011494,-0.142636,0.011190,-0.224195,0.000609,-0.341878


In [143]:
ret_cols = sorted(
    [col for col in df.columns if col.startswith("RET_")],
    key=lambda x: int(x.split("_")[1])
)
volume_cols = sorted(
    [col for col in df.columns if col.startswith("VOLUME_")],
    key=lambda x: int(x.split("_")[1])
)

# expand RET and VOLUME
rets = df[ret_cols].values.ravel()
volumes = df[volume_cols].values.ravel()

# repeat INDUSTRY, INDUSTRY_GROUP, SECTOR, SUB_INDUSTRY for each RET and VOLUME
industry = np.repeat(df["INDUSTRY"].values, len(ret_cols))
industry_group = np.repeat(df["INDUSTRY_GROUP"].values, len(ret_cols))
sector = np.repeat(df["SECTOR"].values, len(ret_cols))
sub_industry = np.repeat(df["SUB_INDUSTRY"].values, len(ret_cols))

# generate day for RET{i} and VOLUME{i}
days = np.tile(np.arange(20,0,-1), len(df))


# generate ID and stock (repeat 20 times)
ids = np.repeat(df["ID"].values, len(ret_cols))
stocks = np.repeat(df["STOCK"].values, len(ret_cols))

df_signature = pd.DataFrame({
    "ID": ids,
    "STOCK": stocks,
    "DAY": days,
    "RET": rets,
    "VOLUME": volumes,
    "INDUSTRY": industry,
    "INDUSTRY_GROUP": industry_group,
    "SECTOR": sector,
    "SUB_INDUSTRY": sub_industry
})

df_signature = df_signature.sort_values(["STOCK", "ID", "DAY"])
df_signature

,ID,STOCK,DAY,RET,VOLUME,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY
19,2377,0,1,-0.019463,-0.291751,37,12,5,94
18,2377,0,2,0.005114,-0.324675,37,12,5,94
17,2377,0,3,-0.001064,-0.351363,37,12,5,94
16,2377,0,4,-0.007689,-0.585497,37,12,5,94
15,2377,0,5,-0.009262,-0.493354,37,12,5,94
...,...,...,...,...,...,...,...,...,...
8371884,416228,5716,16,-0.003120,-0.276386,50,17,7,114
8371883,416228,5716,17,-0.056791,-0.059554,50,17,7,114
8371882,416228,5716,18,0.006046,-0.296746,50,17,7,114
8371881,416228,5716,19,0.024358,-0.189710,50,17,7,114


### Issue RET and VOLUME has outliers, maybe need to drop them

In [144]:
print(df_signature.RET.max())
print(df_signature.RET.min())
print(df_signature.VOLUME.max())
print(df_signature.VOLUME.min())

4.7874880981224415
-2.2663186682994865
1232.1740087603175
-5.6000557832532785


In [145]:
df_signature = df_signature[
    (df_signature['RET'] >= df_signature['RET'].quantile(0.005)) & 
    (df_signature['RET'] <= df_signature['RET'].quantile(0.995)) & 
    (df_signature['VOLUME'] >= df_signature['VOLUME'].quantile(0.005)) & 
    (df_signature['VOLUME'] <= df_signature['VOLUME'].quantile(0.995))
]


Day SIG_3 === 19?? why?

In [146]:
def calculate_signature(df, order=3, sig_col=[]):
    grouped = df.groupby(["STOCK", "ID", 'INDUSTRY', 'INDUSTRY_GROUP', 'SECTOR', 'SUB_INDUSTRY'])
    signatures = []
    keys = []

    for (stock, id_, industry, industry_group, sector, sub_industry), group in grouped:
        group = group.sort_values("DAY", ascending=True)
        path = group[sig_col].values.astype(np.float64)

        base_sig = iisignature.sig(path, order)
        sig = np.insert(base_sig, 0, 1.0) # augmented signature
        signatures.append(sig)
        keys.append((stock, id_, industry, industry_group, sector, sub_industry))
    
    sig_length = iisignature.siglength(len(sig_col), order)
    sig_columns = [f"SIG_{i}" for i in range(sig_length+1)]
    
    df_signature = pd.DataFrame(signatures, columns=sig_columns)
    df_signature[["STOCK", "ID", 'INDUSTRY', 'INDUSTRY_GROUP', 'SECTOR', 'SUB_INDUSTRY']] = keys
    
    # Reorganize columns
    cols_order = ["STOCK", "ID", 'INDUSTRY', 'INDUSTRY_GROUP', 'SECTOR', 'SUB_INDUSTRY'] + sig_columns
    df_signature = df_signature[cols_order]
    
    return df_signature

sig_lst = ["RET", "VOLUME", "DAY"]
df_final = calculate_signature(df_signature, order=3, sig_col=sig_lst)
df_final

,STOCK,ID,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,SIG_0,SIG_1,SIG_2,SIG_3,...,SIG_30,SIG_31,SIG_32,SIG_33,SIG_34,SIG_35,SIG_36,SIG_37,SIG_38,SIG_39
0,0,2377,37,12,5,94,1.0,0.013478,0.428449,19.0,...,16.742766,0.002544,0.102650,1.126432,-0.044290,2.204561,60.680047,-2.201951,-0.087743,1143.166667
1,0,5198,37,12,5,94,1.0,0.011891,-0.218929,19.0,...,-41.360985,0.000541,-0.127537,-0.066829,0.116247,0.381386,7.583428,0.314765,-5.739110,1143.166667
2,0,8017,37,12,5,94,1.0,-0.010962,0.452785,19.0,...,0.668845,0.002929,0.068548,-0.211418,-0.178865,1.542704,34.266897,-2.813371,46.791971,1143.166667
3,0,20826,37,12,5,94,1.0,0.000254,0.657544,19.0,...,16.388365,0.000423,-0.013742,-0.302089,0.057454,3.168119,9.863237,0.936377,92.435108,1143.166667
4,0,33843,37,12,5,94,1.0,0.032979,-1.592417,19.0,...,-298.038860,0.000506,0.007140,0.476700,-0.025107,1.288694,-26.644716,-0.711383,37.252379,1143.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418590,5716,391556,50,17,7,114,1.0,0.044924,0.153095,19.0,...,54.252585,0.006379,-0.095900,0.867096,0.059219,0.437152,-13.880403,3.414761,-12.738529,1143.166667
418591,5716,394490,50,17,7,114,1.0,0.000129,0.108667,19.0,...,5.026164,0.000749,0.041252,-0.583668,-0.018718,0.735698,-12.246397,0.985782,26.834676,1143.166667
418592,5716,400150,50,17,7,114,1.0,-0.013193,0.308652,19.0,...,100.368624,0.003832,-0.358276,1.271780,0.429097,2.279437,-21.898073,-2.078660,-22.758782,1143.166667
418593,5716,403129,50,17,7,114,1.0,0.010215,0.514191,19.0,...,36.957702,0.001754,0.075941,0.602905,-0.020663,1.329850,-10.061456,1.540369,65.915190,1143.166667


In [147]:
df_final = pd.merge(df_final, y_train, on=["ID"], how="inner")
df_final['RET'] = df_final["RET"].astype(int) # Convert RET to binary classification target
df_final

,STOCK,ID,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,SIG_0,SIG_1,SIG_2,SIG_3,...,SIG_31,SIG_32,SIG_33,SIG_34,SIG_35,SIG_36,SIG_37,SIG_38,SIG_39,RET
0,0,2377,37,12,5,94,1.0,0.013478,0.428449,19.0,...,0.002544,0.102650,1.126432,-0.044290,2.204561,60.680047,-2.201951,-0.087743,1143.166667,0
1,0,5198,37,12,5,94,1.0,0.011891,-0.218929,19.0,...,0.000541,-0.127537,-0.066829,0.116247,0.381386,7.583428,0.314765,-5.739110,1143.166667,0
2,0,8017,37,12,5,94,1.0,-0.010962,0.452785,19.0,...,0.002929,0.068548,-0.211418,-0.178865,1.542704,34.266897,-2.813371,46.791971,1143.166667,1
3,0,20826,37,12,5,94,1.0,0.000254,0.657544,19.0,...,0.000423,-0.013742,-0.302089,0.057454,3.168119,9.863237,0.936377,92.435108,1143.166667,1
4,0,33843,37,12,5,94,1.0,0.032979,-1.592417,19.0,...,0.000506,0.007140,0.476700,-0.025107,1.288694,-26.644716,-0.711383,37.252379,1143.166667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418590,5716,391556,50,17,7,114,1.0,0.044924,0.153095,19.0,...,0.006379,-0.095900,0.867096,0.059219,0.437152,-13.880403,3.414761,-12.738529,1143.166667,1
418591,5716,394490,50,17,7,114,1.0,0.000129,0.108667,19.0,...,0.000749,0.041252,-0.583668,-0.018718,0.735698,-12.246397,0.985782,26.834676,1143.166667,1
418592,5716,400150,50,17,7,114,1.0,-0.013193,0.308652,19.0,...,0.003832,-0.358276,1.271780,0.429097,2.279437,-21.898073,-2.078660,-22.758782,1143.166667,0
418593,5716,403129,50,17,7,114,1.0,0.010215,0.514191,19.0,...,0.001754,0.075941,0.602905,-0.020663,1.329850,-10.061456,1.540369,65.915190,1143.166667,1


In [148]:
# df_final.to_csv('signature.csv') # uncomment when exporting data

In [149]:
industry_lst = df_final.INDUSTRY.unique() # 'INDUSTRY'
industry_group_lst = df_final.INDUSTRY_GROUP.unique() # 'INDUSTRY_GROUP'
sector_lst = df_final.SECTOR.unique() # 'SECTOR'
sub_industry_lst = df_final.SUB_INDUSTRY.unique() # 'SUB_INDUSTRY'

In [158]:
param_grid = {"alpha": np.logspace(-3, 3, num=10)}
results = []

for idx, df_grouped in df_final.groupby("SECTOR"):
    # Build feature matrix
    feature_cols = [c for c in df_grouped.columns if c.startswith("SIG_")]
    X = df_grouped[feature_cols].values
    y = np.array(df_grouped["RET"])
    
    # Scale the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Train/validation split
    X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Grid search (5-fold CV)
    gs = GridSearchCV(
        estimator=RidgeClassifier(),
        param_grid=param_grid,
        cv=5,
        scoring="accuracy",
        # scoring="f1",
        n_jobs=-1
    )
    gs.fit(X, y)

    # Evaluate on validation set
    best_alpha = gs.best_params_["alpha"]
    best_model = gs.best_estimator_
    y_val_pred = best_model.predict(X_val_sub)
    acc = accuracy_score(y_val_sub, y_val_pred)
    report = classification_report(y_val_sub, y_val_pred)

    # Store results
    results.append({
        "Grouped_by": idx,
        "Num_Samples": len(df_grouped),
        "Train_Samples": len(X_train_sub),
        "Val_Samples": len(X_val_sub),
        "Best_Alpha": best_alpha,
        "Val_Accuracy": acc,
        "Report": report,
        "Model": best_model
    })

# Summarize
df_results = pd.DataFrame(results)
print(df_results[[
    "Grouped_by", "Num_Samples", "Train_Samples", "Val_Samples",
    "Best_Alpha", "Val_Accuracy"
]])

    Grouped_by  Num_Samples  Train_Samples  Val_Samples  Best_Alpha  \
0            0         6304           5043         1261    0.021544   
1            1        21264          17011         4253    0.004642   
2            2        18967          15173         3794   10.000000   
3            3        55473          44378        11095    0.004642   
4            4        63519          50815        12704  215.443469   
5            5        17295          13836         3459    0.001000   
6            6        55123          44098        11025    0.001000   
7            7        87903          70322        17581    0.004642   
8            8        70843          56674        14169  215.443469   
9            9         5555           4444         1111    0.004642   
10          10        13295          10636         2659    0.001000   
11          11         3054           2443          611    0.021544   

    Val_Accuracy  
0       0.518636  
1       0.525982  
2       0.512124  


In [159]:
df_results

,Grouped_by,Num_Samples,Train_Samples,Val_Samples,Best_Alpha,Val_Accuracy,Report,Model
0,0,6304,5043,1261,0.021544,0.518636,precision recall f1-score ...,RidgeClassifier(alpha=np.float64(0.02154434690...
1,1,21264,17011,4253,0.004642,0.525982,precision recall f1-score ...,RidgeClassifier(alpha=np.float64(0.00464158883...
2,2,18967,15173,3794,10.000000,0.512124,precision recall f1-score ...,RidgeClassifier(alpha=np.float64(10.0))
3,3,55473,44378,11095,0.004642,0.519063,precision recall f1-score ...,RidgeClassifier(alpha=np.float64(0.00464158883...
4,4,63519,50815,12704,215.443469,0.520860,precision recall f1-score ...,RidgeClassifier(alpha=np.float64(215.443469003...
5,5,17295,13836,3459,0.001000,0.533391,precision recall f1-score ...,RidgeClassifier(alpha=np.float64(0.001))
6,6,55123,44098,11025,0.001000,0.516190,precision recall f1-score ...,RidgeClassifier(alpha=np.float64(0.001))
7,7,87903,70322,17581,0.004642,0.521984,precision recall f1-score ...,RidgeClassifier(alpha=np.float64(0.00464158883...
8,8,70843,56674,14169,215.443469,0.521561,precision recall f1-score ...,RidgeClassifier(alpha=np.float64(215.443469003...
9,9,5555,4444,1111,0.004642,0.526553,precision recall f1-score ...,RidgeClassifier(alpha=np.float64(0.00464158883...
